In [23]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


# pd.set_option('display.max_colwidth', None)

In [24]:
similarity_path = "../data/processed/vis_dataset/similarity_combined.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html"

In [25]:
dataset = pd.read_csv(dataset_path)
dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
1354,InfoVis,2016,Evaluating the Impact of Binning 2D Scalar Fields,10.1109/TVCG.2016.2599106,1354,"[872, 635, 1195]","[1384, 1422]",http://dx.doi.org/10.1109/TVCG.2016.2599106,431,440,...,The expressiveness principle for visualization...,Lace M. K. Padilla;P. Samuel Quinan;Miriah D. ...,Lace Padilla;P. Samuel Quinan;Miriah Meyer;Sar...,"Department of Psychology, University of Utah;U...",10.1109/TVCG.2011.175;10.1109/TVCG.2015.246775...,Geographic/Geospatial Visualization;Qualitativ...,11.0,16.0,54.0,NaN
677,VAST,2011,Network-based visual analysis of tabular data,10.1109/VAST.2011.6102440,677,"[257, 325, 550, 267, 108, 604, 351]","[864, 913, 953, 956, 1026, 1088, 1283]",http://dx.doi.org/10.1109/VAST.2011.6102440,41,50,...,Tabular data are pervasive. Although tables of...,Zhicheng Liu;Shamkant B. Navathe;John T. Stasko,Zhicheng Liu;Shamkant B. Navathe;John T. Stasko,"Georgia Institute of Technology, USA;Georgia I...",10.1109/TVCG.2006.122;10.1109/TVCG.2010.177;10...,NaN,35.0,20.0,37.0,HM


In [26]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken of the uncertainties in parameter values arising from inevitab

In [27]:
df = pd.read_csv(similarity_path)
df

,id,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,1742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1743,1743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1744,1744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.0)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,0,238,1.0
1,0,244,1.0
2,0,456,1.0
3,0,512,1.0
4,0,550,1.0
...,...,...,...
60213,1611,1612,2.0
60214,1611,1614,1.0
60215,1611,1615,2.0
60216,1614,1615,1.0


## Plot using Pyvis

In [29]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [30]:
!open /Users/chaupham/opt/anaconda3/envs/py38_env/lib/python3.8/site-packages/pyvis

In [34]:
thres = 6
df_out_thres = df_out[df_out["value"] >= thres]
plotting_all_nodes = False

file_name= output_path
g = net.Network(height='95%', width='70%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')
# g.set_template("/Users/chaupham/opt/anaconda3/envs/py38_env/lib/python3.8/site-packages/pyvis/templates/template_2.html")

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
for e in edge_list:
    g.add_edge(e[0], e[1], value = e[2], title=str(e[2])) #arrows = 'to')
    
# g.show_buttons(filter_=["physics"])
# g.show_buttons()

g.show(file_name)

In [35]:
!open $file_name

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [44]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres


edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Title"].values[0])
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges.csv", index=False)

# port=8050

# while True:
#     try:
#         Jaal(edges, nodes).plot(port=port)
#     except:
#         port+=1


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


(1746, 4)
(294, 4)


In [45]:
edges

,from,to,value,label
142,6,351,7.0,7
353,21,238,6.0,6
908,50,60,6.0,6
1174,59,108,7.0,7
1180,59,148,9.0,9
...,...,...,...,...
59726,1527,1591,7.0,7
60004,1562,1616,6.0,6
60106,1582,1589,6.0,6
60162,1591,1592,6.0,6


In [46]:
nodes

,id,having_edge,conference,Title
6,6,True,InfoVis,Visualizing the non-visual: spatial analysis a...
21,21,True,InfoVis,On the semantics of interactive visualizations
50,50,True,InfoVis,The structure of the information visualization...
59,59,True,InfoVis,Similarity clustering of dimensions for an enh...
60,60,True,InfoVis,An operator interaction framework for visualiz...
...,...,...,...,...
1592,1592,True,InfoVis,Charticulator: Interactive Construction of Bes...
1598,1598,True,InfoVis,NLIZE: A Perturbation-Driven Visual Interrogat...
1606,1606,True,InfoVis,A Framework for Creative Visualization-Opportu...
1607,1607,True,InfoVis,At a Glance: Pixel Approximate Entropy as a Me...


In [36]:
nodes[nodes.id==1606].Title.values[0]

'A Framework for Creative Visualization-Opportunities Workshops'

In [56]:
edges[(edges["to"] == 755) & (edges["from"]==456)]

,from,to,value,label
18741,456,755,21.0,21
